In [1]:

import os
import numpy as np
import pandas as pd
import cv2 as cv
# import torch
# import utils
# display = utils.notebook_init()  # checks

ROOT = '/home/san/code/optimaize/Yolov5_StrongSORT_OSNet/'
%cd $ROOT

/home/san/code/optimaize/Yolov5_StrongSORT_OSNet


In [2]:
# Video procesing
input_  = os.path.join(ROOT, 'data', 'inovelec', 'videos', 'setup2.mp4')

base    = os.path.basename(input_)
split   = os.path.splitext(base)
preproc = os.path.join(os.path.dirname(input_), split[0] + '_short' + split[1])


In [ ]:
# CROP, ROTATE AND CUT ORIGINAL VIDEO:
# !ffmpeg -i $input_ -ss 00:00:28.00 -t 00:01:10  -vsync 0 -vf "rotate=+5 *PI/180, crop=in_w*4/5:in_h*2/3:in_w/5:in_h/3"  -y $preproc
!ffmpeg -i $input_ -ss 00:19:58.00 -t 00:20:18  -vsync 0 -vf "rotate=+5 *PI/180"  -y $preproc

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
# VIEW RESULT
!ffplay $preproc
# !ffplay $input_

In [11]:
# RUN TRACK ALGORITHM
!python track.py --source $preproc --yolo-weights weights/crowdhuman_yolov5m.pt --classes 0  --show-vid --save-conf --save-txt --save-vid # tracks persons, only

/home/san/code/optimaize/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
track: yolo_weights=['weights/crowdhuman_yolov5m.pt'], strong_sort_weights=/home/san/code/optimaize/Yolov5_StrongSORT_OSNet/weights/osnet_x0_25_msmt17.pt, config_strongsort=strong_sort/configs/strong_sort.yaml, source=/home/san/code/optimaize/Yolov5_StrongSORT_OSNet/data/inovelec/videos/setup2_short.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=True, save_txt=True, save_conf=True, save_crop=False, save_vid=True, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False
YOLOv5 🚀 2022-7-22 Python-3.9.7 torch-1.12.0+cu102 CPU

Fusing layers... 
Model summa

In [3]:
# Create dataframe from txt files

base=os.path.basename(preproc)
path = os.path.join('.', 'runs', 'track', 'exp4', 'tracks', os.path.splitext(base)[0]+'.txt') #runs/detect/inovelec
path

ino = pd.read_csv(path, sep=' ', index_col=False, names=['frame', 'id', 'xini', 'yini', 'xfin', 'yfin', 'score1', 'score2', 'score3', 'class'])

#ino = pd.DataFrame(stack, columns = ['frame', 'id', 'xini', 'yini', 'xfin', 'yfin', 'score1', 'score2', 'score3', 'class'])
ino.drop(columns=['score1', 'score2', 'score3'], inplace=True)
# ino = ino.astype({'frame':'int','class':'int', 'xini':'float', 'yini':'float', 'xfin':'float', 'yfin':'float', 'score':'float'})
ino.set_index(['id', 'frame'], inplace=True)
ino

,,xini,yini,xfin,yfin,class
id,frame,,,,,
1,3,480,339,62,130,0
2,3,1071,343,46,135,0
3,3,918,371,69,109,0
4,3,1103,360,47,115,0
5,3,545,340,38,104,0
...,...,...,...,...,...,...
2,697,434,341,53,135,0
3,697,904,370,63,136,0
4,697,1099,358,49,85,0


In [5]:
# Plot people centroids in the video to check scale and axes

base  = os.path.basename(preproc)
split = os.path.splitext(base)
out = os.path.join(os.path.dirname(preproc), split[0] + '_tag' + split[1])

cap = cv.VideoCapture(preproc)



radius    = 5
red     = (0, 0, 255)
blue    = (255, 0, 0)
green   = (0, 255, 0)
white   = (255, 255, 255)
thickness = -1


# The tracker do not make predictions on the first two frames so I throw away them end get the third
# No need to initialize i as it output loop equal to 2
operator_id = 2

for i in range(3):
    ret, frame = cap.read()    

i=3

centroid_prev = (ino.loc[operator_id,i]['xini']+ino.loc[operator_id,i]['xfin']//2, 
                 ino.loc[operator_id,i]['yini'])
    
    
print(frame.shape)
height, width, channels = frame.shape
# fps = 25 
# write new video
# fourcc = cv.VideoWriter_fourcc(*'mp4v')
# out    = cv.VideoWriter(out_path, fourcc, fps, (w, h))

while(ret):

    frame  = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame  = cv.cvtColor(frame, cv.COLOR_GRAY2RGB)

    # print(logical.sum)
    try:
        centroid = (ino.loc[operator_id,i]['xini']+ino.loc[operator_id,i]['xfin']//2, 
                    ino.loc[operator_id,i]['yini'])
    except:
        centroid = centroid_prev

    centroid_prev = centroid # if raise exception, copy last valid value (operator stays in the same position)
    
        
    frame  = cv.circle(frame, centroid, radius, red, thickness)
    # center_of_circle = box['right_hand_center'][i]    
    # frame  = cv.circle(frame, center_of_circle, radius, blue, thickness)
        
    cv.imshow('frame', frame)
    # out.write(frame)

    if cv.waitKey(25) & 0xFF == ord('q'):
        break

    
    i += 1
    
    ret, frame = cap.read()

    

# out.release()
print('released.....................................')
cap.release()
cv.destroyAllWindows()

(720, 1280, 3)
released.....................................


In [14]:
# ORTHOGRAPHICAL PROJECTION

cap = cv.VideoCapture(preproc)

ret, frame = cap.read()
print(frame.shape)


# pt_A = [504,358]
# pt_B = [8, 482]
# pt_C = [1263,490]
# pt_D = [878,355]

pt_A = [632,324]
pt_B = [177,525]
pt_C = [1274,428]
pt_D = [760,313]

red     = (0, 0, 255)
blue    = (255, 0, 0)
green   = (0, 255, 0)
white   = (255, 255, 255)
radius    = 5
thickness = -1
line_thickness = 2


# frame  = cv.circle(frame, pt_A, radius, red, thickness)
# frame  = cv.circle(frame, pt_B, radius, blue, thickness)
# frame  = cv.circle(frame, pt_C, radius, white, thickness)
# frame  = cv.circle(frame, pt_D, radius, green, thickness)

# frame = cv.line(frame, pt_A, pt_B, (0, 255, 0), thickness=line_thickness)
# frame = cv.line(frame, pt_B, pt_C, (0, 255, 0), thickness=line_thickness)
# frame = cv.line(frame, pt_C, pt_D, (0, 255, 0), thickness=line_thickness)
# frame = cv.line(frame, pt_D, pt_A, (0, 255, 0), thickness=line_thickness)

cv.imwrite(ROOT + 'notebooks/workarea.jpg', frame)




# center_of_circle = box['right_hand_center'][i]    
# frame  = cv.circle(frame, center_of_circle, radius, blue, thickness)

# cv.imshow('frame', frame)






# # Here, I have used L2 norm. You can use L1 also.
# width_AD = np.sqrt(((pt_A[0] - pt_D[0]) ** 2) + ((pt_A[1] - pt_D[1]) ** 2))
# width_BC = np.sqrt(((pt_B[0] - pt_C[0]) ** 2) + ((pt_B[1] - pt_C[1]) ** 2))
# maxWidth = max(int(width_AD), int(width_BC))


# height_AB = np.sqrt(((pt_A[0] - pt_B[0]) ** 2) + ((pt_A[1] - pt_B[1]) ** 2))
# height_CD = np.sqrt(((pt_C[0] - pt_D[0]) ** 2) + ((pt_C[1] - pt_D[1]) ** 2))
# maxHeight = max(int(height_AB), int(height_CD))

# input_pts = np.float32([pt_A, pt_B, pt_C, pt_D])
# output_pts = np.float32([[0, 0],
#                         [0, maxHeight - 1],
#                         [maxWidth - 1, maxHeight - 1],
#                         [maxWidth - 1, 0]])

# # Compute the perspective transform M
# M = cv.getPerspectiveTransform(input_pts,output_pts)


# out = cv.warpPerspective(frame,M,(maxWidth, maxHeight),flags=cv.INTER_LINEAR)


# cv.imshow("original_img",frame)
# cv.imshow("result",out)
# cv.waitKey(0)
# cv.destroyAllWindows()


(720, 1280, 3)


True

In [16]:

cap = cv.VideoCapture(input_)
frame_number = 53046
cap.set(1,frame_number); # Where frame_no is the frame you want
ret, frame = cap.read() # Read the frame
cv.imwrite(ROOT + 'notebooks/workarea.jpg', frame)


True